<div style="background-color:dodgerblue;height:100px;"><br>
<center><h1><font style="color:white">The Path to Self-Discovery: Build Structure, Get Meaning</font></h1></center>
</div>

# Looking Back at the Past

In order to move forward, we need to remember where we have been.

When we left off, we had a list of records based off of our sample data.

** As we said in the last lesson, all of this parsing is based off of assumptions.  Changes were made to the function below to account for some new conditions.**

In [1]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(open('./My Activity - Sample.html', 'r', encoding='iso8859-1').read(), 'html.parser')

In [2]:
import re

location_pattern = '\?q=([^\s]+?),([^\s]+)'

def parse_record(element):
    ### need to properly parse out the search target info.  first line, split on the visited or serached for, take the text from rest
    section1 = element.find("div", "content-cell")
    
    if section1.find(text=re.compile('Visited|Searched')):
        section1_strings = list(section1.strings)
        activity_type = None
        target = None
        datestamp = None
        
        if len(section1_strings)==3:
            activity_type, target, datestamp = section1_strings
        elif len(section1_strings)==2:
            activity_type, target = section1_strings[0].split('\xa0')
            datestamp = section1_strings[-1]
        else:
            print('---Unknown----')
            section1_strings
            return None
        activity_type = activity_type.replace('\xa0', '').strip()

        section2 = element.select("div.content-cell.mdl-typography--caption")[0]
        section_strings = list(section2.strings)
        product = section_strings[1].replace('\u2003','').strip()
        lat = None
        lon = None

        if 'Locations:' in section_strings:
            location_url = section2.find("a",{"href":re.compile("https://google.com/maps\?q=")}).text
            match = re.search(location_pattern, location_url)
            if match:
                lat, lon = match.groups()

        data = {
            'Activity Type': re.sub('[^A-Za-z\s]', '', activity_type),
            'Target': target,
            'Datestamp': datestamp,
            'Product': re.sub('[^A-Za-z\s]', '', product),
            'Lat':lat,
            'Lon':lon
        }

        return data
    
    else:
        return None

my_records = []

for record in soup.find_all('div', 'outer-cell'):
    new_record = parse_record(record)
    if new_record:
        my_records.append(new_record)
    
for record in my_records:
    print(record)

{'Lon': None, 'Target': 'https://productforums.google.com/forum/', 'Activity Type': 'Visited', 'Datestamp': 'Jan 31, 2018, 10:54:50 PM', 'Lat': None, 'Product': 'Search'}
{'Lon': None, 'Target': 'http://www.adobe.com/creativecloud.html', 'Activity Type': 'Visited', 'Datestamp': 'Feb 8, 2017, 12:32:39 AM', 'Lat': None, 'Product': 'Search'}
{'Lon': '-80.186310', 'Target': 'adobe creative cloud', 'Activity Type': 'Searched for', 'Datestamp': 'Feb 8, 2017, 12:32:36 AM', 'Lat': '25.800819', 'Product': 'Search'}


## Exercise 1

We ran this function over a sample dataset, but now we need to do it over the whole shabang!  Using everything you learned from the last lesson (and cheating with the example above), parse the records out of the full data file.  This is where you can use your own, or the dummy dataset we provided!

In [3]:
soup = BeautifulSoup(open('./My Activity - WIDS.html', 'r', encoding='iso8859-1').read(), 'html.parser')

full_records = []

error_count = 0

for element in soup.find_all('div', 'outer-cell'):
    temp = parse_record(element)
    if temp:
        full_records.append(temp)
    else:
        error_count += 1


In [4]:
print(len(full_records))
print(full_records[0:10])

1716
[{'Lon': None, 'Target': 'https://stackoverflow.com/questions/2299022/selecting-data-from-two-different-servers-in-sql-server', 'Activity Type': 'Visited', 'Datestamp': 'Aug 31, 2017, 9:54:24 PM', 'Lat': None, 'Product': 'Search'}, {'Lon': None, 'Target': 'https://stackoverflow.com/questions/12901979/sql-query-across-multiple-sql-servers', 'Activity Type': 'Visited', 'Datestamp': 'Aug 31, 2017, 9:54:06 PM', 'Lat': None, 'Product': 'Search'}, {'Lon': '-77.001811', 'Target': 'sql query multiple servers', 'Activity Type': 'Searched for', 'Datestamp': 'Aug 31, 2017, 9:54:03 PM', 'Lat': '38.913383', 'Product': 'Search'}, {'Lon': None, 'Target': 'https://social.msdn.microsoft.com/forums/sqlserver/en-us/191b11ab-2fec-21c9-a0e1-92eb11011cc9/queries-from-multiple-databases-on-the-same-server?forum=transactsql', 'Activity Type': 'Visited', 'Datestamp': 'Aug 31, 2017, 9:53:58 PM', 'Lat': None, 'Product': 'Search'}, {'Lon': '-77.001803', 'Target': 'sql query multiple databases', 'Activity Typ

Now we have a looot more records to deal with.  And they certainly aren't the easiest to read or navigate.  What we really want is a pretty, matrix structure, so that we can look at both the rows and columns.  A list of dictionaries is not quite that.  So what do we do?

# Introducing Pandas

Pandas is a Python library that makes it easy to load and manipulate 1-3 dimensional data.  These objects are called:
* 1-D: Series
* 2-D: DataFrame
* 3-D: Panel

We can generate these objects from:
* CSV
* Excel
* HTML
* List of dictionaries
* List of lists
* ..and more

You should read all about how to create DataFrames, but since we have already built our data, the DataFrame constructor knows how to take our list of dictionaries in and make it pretty!

In [5]:
import pandas as pd

In [6]:
df = pd.DataFrame(full_records)

We can preview this DataFrame using the `head` function.

In [7]:
df.head()

,Activity Type,Datestamp,Lat,Lon,Product,Target
0,Visited,"Aug 31, 2017, 9:54:24 PM",None,None,Search,https://stackoverflow.com/questions/2299022/se...
1,Visited,"Aug 31, 2017, 9:54:06 PM",None,None,Search,https://stackoverflow.com/questions/12901979/s...
2,Searched for,"Aug 31, 2017, 9:54:03 PM",38.913383,-77.001811,Search,sql query multiple servers
3,Visited,"Aug 31, 2017, 9:53:58 PM",None,None,Search,https://social.msdn.microsoft.com/forums/sqlse...
4,Searched for,"Aug 31, 2017, 9:53:54 PM",38.913384,-77.001803,Search,sql query multiple databases


In [8]:
df.head(2)

,Activity Type,Datestamp,Lat,Lon,Product,Target
0,Visited,"Aug 31, 2017, 9:54:24 PM",None,None,Search,https://stackoverflow.com/questions/2299022/se...
1,Visited,"Aug 31, 2017, 9:54:06 PM",None,None,Search,https://stackoverflow.com/questions/12901979/s...


Or we can grab the end of our DataFrame with `tail`.

In [9]:
df.tail(3)

,Activity Type,Datestamp,Lat,Lon,Product,Target
1713,Searched for,"Aug 1, 2017, 8:46:01 AM",38.916431,-76.997355,Search,sharepoint 1020 give everyone read access
1714,Searched for,"Aug 1, 2017, 8:08:11 AM",38.930142,-77.009456,Search,chapel hill events
1715,Searched for,"Aug 1, 2017, 8:00:52 AM",38.918279,-77.00027,Search,directions to work


# Starting Simple

Right off the bat, pandas gives us some quick ways to scope our DataFrame.  Let's explore some of these basic operations.

In [10]:
len(df)

1716

In [11]:
df.size

10296

In [12]:
df.shape

(1716, 6)

In [13]:
df.shape[0]*df.shape[1]

10296

In [14]:
len(df.columns)

6

In [15]:
df.columns

Index(['Activity Type', 'Datestamp', 'Lat', 'Lon', 'Product', 'Target'], dtype='object')

In [16]:
df.describe()

,Activity Type,Datestamp,Lat,Lon,Product,Target
count,1716,1716,950,950,1716,1716
unique,2,1704,136,119,1,1591
top,Searched for,"Aug 2, 2017, 9:00:17 AM",39.105718,-77.15755,Search,directions to home
freq,1000,2,388,388,1716,17


In [17]:
df.count()

Activity Type    1716
Datestamp        1716
Lat               950
Lon               950
Product          1716
Target           1716
dtype: int64

In [18]:
df['Activity Type'].count()

1716

# The Data Type Cleanse

We can list the data types of each column using the `dtypes` command.

In [19]:
df.dtypes

Activity Type    object
Datestamp        object
Lat              object
Lon              object
Product          object
Target           object
dtype: object

In [20]:
df.head()

,Activity Type,Datestamp,Lat,Lon,Product,Target
0,Visited,"Aug 31, 2017, 9:54:24 PM",None,None,Search,https://stackoverflow.com/questions/2299022/se...
1,Visited,"Aug 31, 2017, 9:54:06 PM",None,None,Search,https://stackoverflow.com/questions/12901979/s...
2,Searched for,"Aug 31, 2017, 9:54:03 PM",38.913383,-77.001811,Search,sql query multiple servers
3,Visited,"Aug 31, 2017, 9:53:58 PM",None,None,Search,https://social.msdn.microsoft.com/forums/sqlse...
4,Searched for,"Aug 31, 2017, 9:53:54 PM",38.913384,-77.001803,Search,sql query multiple databases


We can also change a columns data type, or cast it as a new type, using the `astype` command.

In [21]:
df['Lat'] = df['Lat'].astype(float)
df['Lon'] = df['Lon'].astype(float)
df.dtypes

Activity Type     object
Datestamp         object
Lat              float64
Lon              float64
Product           object
Target            object
dtype: object

We stil would like to get the Datestamp column in a more friendly format, though.  The way it is now could trip us up, because calculations will be performed as if the column is a string.

In [22]:
df['Datestamp'].min()

'Aug 1, 2017, 10:00:21 PM'

In [23]:
df.Datestamp.max()

'Aug 9, 2017, 8:30:19 AM'

Luckily, pandas `to_datetime` command easily parses our datestamps without us having to do any date formatting!

In [24]:
df['Datestamp'] = pd.to_datetime(df['Datestamp'])

In [25]:
df['Datestamp'].min()

Timestamp('2017-08-01 08:00:52')

In [26]:
df['Datestamp'].max()

Timestamp('2017-08-31 21:54:24')

# Filtering Out the Muck

Pandas also allows us to easily filter our data!  We can use some really simple syntax for this.

In [35]:
df.head()

,Activity Type,Datestamp,Lat,Lon,Product,Target
0,Visited,2017-08-31 21:54:24,NaN,NaN,Search,https://stackoverflow.com/questions/2299022/se...
1,Visited,2017-08-31 21:54:06,NaN,NaN,Search,https://stackoverflow.com/questions/12901979/s...
2,Searched for,2017-08-31 21:54:03,38.913383,-77.001811,Search,sql query multiple servers
3,Visited,2017-08-31 21:53:58,NaN,NaN,Search,https://social.msdn.microsoft.com/forums/sqlse...
4,Searched for,2017-08-31 21:53:54,38.913384,-77.001803,Search,sql query multiple databases


In [36]:
df[(df['Target']=='sql query multiple servers')]

,Activity Type,Datestamp,Lat,Lon,Product,Target
2,Searched for,2017-08-31 21:54:03,38.913383,-77.001811,Search,sql query multiple servers


In [29]:
temp_df = df[~df['Lat'].isnull()].head()
temp_df.head(1).squeeze()

Activity Type                  Searched for
Datestamp               2017-08-31 21:54:03
Lat                                 38.9134
Lon                                -77.0018
Product                              Search
Target           sql query multiple servers
Name: 2, dtype: object

# Asking Questions of Our Data Using Pandas

Now, armed with this small set of tools, let's use pandas to answer some questions we would commonly have about our data!

## Exercise 2
How many rows of data are in the dataset?

In [30]:
len(df)

1716

## Exercise 3

How many search activities searched for 'tsql multiple with statements'?

In [37]:
len(df[df['Target'] == 'tsql multiple with statements'])

1

## Exercise 4

What is the median latitude in the dataset?

In [39]:
df['Lat'].median()

39.105443

## Exercise 5

What are the average latitude and longitude?

In [40]:
df[['Lat', 'Lon']].mean()

Lat    38.945140
Lon   -77.125636
dtype: float64

## <i> Challenge Question</i>: Exercise 6 

What was the top search made on August 1, 2017?

In [47]:
df[(df['Datestamp']>pd.to_datetime('2017-08-01')) & (df['Datestamp']<pd.to_datetime('2017-08-02'))]['Target'].mode()

0    flights from  to  on aug 21, 1021 returning au...
dtype: object